In [1]:
import numpy as np
import os
import pandas as pd
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import datetime
from os.path import isfile, join
from sys import getsizeof
import glob

from random import *

from sklearn import preprocessing
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import StratifiedKFold

from pickle import dump
import joblib

# from keras import backend as K
import tensorflow.keras as k
from tensorflow.keras import utils, models, layers, optimizers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

import tensorflow as tf

In [2]:
# load dataset
df = pd.read_csv("D:/Revi/Courses/TMLC/Projects/2. Deep Learning/4. DMSP Particle Precipitate/Dataset/AI_Ready_DMSP_Data.csv")

In [3]:
df.columns.to_list()

['Datetimes',
 'SC_AACGM_LAT',
 'SC_AACGM_LTIME',
 'ELE_TOTAL_ENERGY_FLUX',
 'ELE_TOTAL_ENERGY_FLUX_STD',
 'ELE_AVG_ENERGY',
 'ELE_AVG_ENERGY_STD',
 'ID_SC',
 'sin_ut',
 'cos_ut',
 'sin_doy',
 'cos_doy',
 'sin_SC_AACGM_LTIME',
 'cos_SC_AACGM_LTIME',
 'F107',
 'Bz',
 'By',
 'Bx',
 'AE',
 'AL',
 'AU',
 'SymH',
 'PC',
 'vsw',
 'vx',
 'psw',
 'borovsky',
 'newell',
 'F107_6hr',
 'Bz_6hr',
 'By_6hr',
 'Bx_6hr',
 'AE_6hr',
 'AL_6hr',
 'AU_6hr',
 'SymH_6hr',
 'PC_6hr',
 'vsw_6hr',
 'vx_6hr',
 'psw_6hr',
 'borovsky_6hr',
 'newell_6hr',
 'F107_5hr',
 'Bz_5hr',
 'By_5hr',
 'Bx_5hr',
 'AE_5hr',
 'AL_5hr',
 'AU_5hr',
 'SymH_5hr',
 'PC_5hr',
 'vsw_5hr',
 'vx_5hr',
 'psw_5hr',
 'borovsky_5hr',
 'newell_5hr',
 'F107_3hr',
 'Bz_3hr',
 'By_3hr',
 'Bx_3hr',
 'AE_3hr',
 'AL_3hr',
 'AU_3hr',
 'SymH_3hr',
 'PC_3hr',
 'vsw_3hr',
 'vx_3hr',
 'psw_3hr',
 'borovsky_3hr',
 'newell_3hr',
 'F107_1hr',
 'Bz_1hr',
 'By_1hr',
 'Bx_1hr',
 'AE_1hr',
 'AL_1hr',
 'AU_1hr',
 'SymH_1hr',
 'PC_1hr',
 'vsw_1hr',
 'vx_1hr',


In [4]:
df = df.set_index('Datetimes')
df.index = pd.to_datetime(df.index)

In [5]:
# Choose columns to drop
cols_to_drop = [c for c in df.columns if ('1min' in c) | ('3min' in c) | ('4min' in c) | ('_5min'in c) | ('15min' in c)| ('5hr' in c) | ('F107_' in c) | ('vx' in c) | ('AE' in c) | ('newell' in c) | ('borovsky' in c) | ('STD' in c) | ('AVG' in c) | ('SC_AACGM_LTIME'==c)]

print(cols_to_drop)

['SC_AACGM_LTIME', 'ELE_TOTAL_ENERGY_FLUX_STD', 'ELE_AVG_ENERGY', 'ELE_AVG_ENERGY_STD', 'AE', 'vx', 'borovsky', 'newell', 'F107_6hr', 'AE_6hr', 'vx_6hr', 'borovsky_6hr', 'newell_6hr', 'F107_5hr', 'Bz_5hr', 'By_5hr', 'Bx_5hr', 'AE_5hr', 'AL_5hr', 'AU_5hr', 'SymH_5hr', 'PC_5hr', 'vsw_5hr', 'vx_5hr', 'psw_5hr', 'borovsky_5hr', 'newell_5hr', 'F107_3hr', 'AE_3hr', 'vx_3hr', 'borovsky_3hr', 'newell_3hr', 'F107_1hr', 'AE_1hr', 'vx_1hr', 'borovsky_1hr', 'newell_1hr', 'F107_45min', 'AE_45min', 'vx_45min', 'borovsky_45min', 'newell_45min', 'F107_30min', 'AE_30min', 'vx_30min', 'borovsky_30min', 'newell_30min', 'F107_15min', 'Bz_15min', 'By_15min', 'Bx_15min', 'AE_15min', 'AL_15min', 'AU_15min', 'SymH_15min', 'PC_15min', 'vsw_15min', 'vx_15min', 'psw_15min', 'borovsky_15min', 'newell_15min', 'F107_10min', 'AE_10min', 'vx_10min', 'borovsky_10min', 'newell_10min', 'F107_5min', 'Bz_5min', 'By_5min', 'Bx_5min', 'AE_5min', 'AL_5min', 'AU_5min', 'SymH_5min', 'PC_5min', 'vsw_5min', 'vx_5min', 'psw_5min'

In [6]:
df = df.drop(columns=cols_to_drop)

In [7]:
# List the new set of columns
df.columns.to_list()

['SC_AACGM_LAT',
 'ELE_TOTAL_ENERGY_FLUX',
 'ID_SC',
 'sin_ut',
 'cos_ut',
 'sin_doy',
 'cos_doy',
 'sin_SC_AACGM_LTIME',
 'cos_SC_AACGM_LTIME',
 'F107',
 'Bz',
 'By',
 'Bx',
 'AL',
 'AU',
 'SymH',
 'PC',
 'vsw',
 'psw',
 'Bz_6hr',
 'By_6hr',
 'Bx_6hr',
 'AL_6hr',
 'AU_6hr',
 'SymH_6hr',
 'PC_6hr',
 'vsw_6hr',
 'psw_6hr',
 'Bz_3hr',
 'By_3hr',
 'Bx_3hr',
 'AL_3hr',
 'AU_3hr',
 'SymH_3hr',
 'PC_3hr',
 'vsw_3hr',
 'psw_3hr',
 'Bz_1hr',
 'By_1hr',
 'Bx_1hr',
 'AL_1hr',
 'AU_1hr',
 'SymH_1hr',
 'PC_1hr',
 'vsw_1hr',
 'psw_1hr',
 'Bz_45min',
 'By_45min',
 'Bx_45min',
 'AL_45min',
 'AU_45min',
 'SymH_45min',
 'PC_45min',
 'vsw_45min',
 'psw_45min',
 'Bz_30min',
 'By_30min',
 'Bx_30min',
 'AL_30min',
 'AU_30min',
 'SymH_30min',
 'PC_30min',
 'vsw_30min',
 'psw_30min',
 'Bz_10min',
 'By_10min',
 'Bx_10min',
 'AL_10min',
 'AU_10min',
 'SymH_10min',
 'PC_10min',
 'vsw_10min',
 'psw_10min']

In [8]:
print('number of input features total = {}'.format(df.shape[1]))

number of input features total = 73


In [9]:
# Define the threshold above which to remove energy flux values
eflux_threshold_quantile = 99.995
eflux_threshold = .99995

# Define the conversion factor from eV to erg
erg_eV_factor = (1.60218e-12)

In [10]:
threshold_drop_val = np.percentile(df['ELE_TOTAL_ENERGY_FLUX'],eflux_threshold_quantile)
df['ELE_TOTAL_ENERGY_FLUX'].values < threshold_drop_val 

array([ True,  True,  True, ...,  True,  True,  True])

In [11]:
# Remove outliers
df_LTIMEs = [[]]
df_STDs = [[]]
print(df.shape)
threshold_drop_val = np.percentile(df['ELE_TOTAL_ENERGY_FLUX'],eflux_threshold_quantile)
print('... dropping rows where total electron energy flux is greater than {:.4E} [eV/cm2/s/ster]'.format(threshold_drop_val))
print('--------> number dropped = {}'.format(len( np.argwhere(df['ELE_TOTAL_ENERGY_FLUX'].values > threshold_drop_val) )))
#df_LTIMEs = df_LTIMEs[ (df['ELE_TOTAL_ENERGY_FLUX'].values < threshold_drop_val) ]
#df_STDs = df_STDs[ (df['ELE_TOTAL_ENERGY_FLUX'].values < threshold_drop_val) ]
df = df[ (df['ELE_TOTAL_ENERGY_FLUX'].values < threshold_drop_val) ]
print(df.shape)

(1945887, 73)
... dropping rows where total electron energy flux is greater than 7.3673E+13 [eV/cm2/s/ster]
--------> number dropped = 98
(1945789, 73)


In [26]:
df_tmp = df.copy()

In [27]:
df = df.sample(1000)
df.shape

(1000, 73)

In [28]:
# Separate training and testing data
mask_val = [(df.index.year == 2010) & (df['ID_SC'].values==16)]
df_val = df[mask_val[0]].copy(deep=True)
df_train = df.copy(deep=True).drop( df.index[mask_val[0]])
print('validation data shape = {}'.format(df_val.shape))
print('train data shape = {}'.format(df_train.shape))
print('NOTE: we have called the withheld data *validation* data here')

validation data shape = (26, 73)
train data shape = (974, 73)
NOTE: we have called the withheld data *validation* data here


In [29]:
# Construct X and y
feature_cols = [c for c in df.columns if not 'ELE' in c]

In [30]:
from sklearn.preprocessing import RobustScaler

In [31]:
X_val = df_val[feature_cols].copy(deep=True)
y_val = df_val['ELE_TOTAL_ENERGY_FLUX'].copy(deep=True)
X_train = df_train[feature_cols].copy(deep=True)
y_train = df_train['ELE_TOTAL_ENERGY_FLUX'].copy(deep=True)
scaler_X = RobustScaler()
scaler_X = scaler_X.fit(X_train.values)
X_val_scaled = scaler_X.transform(X_val.values)
X_train_scaled = scaler_X.transform(X_train.values)

In [32]:
# The units of the energy flux are eV/cm2/s/ster
#   --> To remove the 'steradian' multiply by PI (see: https://agupubs.onlinelibrary.wiley.com/doi/epdf/10.1002/2016JA023339)
y_train = y_train * np.pi
y_val = y_val * np.pi
print('NOTE: removing the steradian from the DMSP data BEFORE training')

NOTE: removing the steradian from the DMSP data BEFORE training


In [33]:
# Create targets in erg/cm2/s units to explore target in different units
y_train_erg = y_train.copy(deep=True) * (1.60218e-12)
y_val_erg = y_val.copy(deep=True) * (1.60218e-12)

In [34]:
# Create targets in log10(eV/cm2/s) units to explore target in different units
y_train[y_train == 0] = 0.00001
y_val[y_val == 0] = 0.00001
y_train_log = np.log10(y_train.copy(deep=True))
y_val_log = np.log10(y_val.copy(deep=True))

In [40]:
#Create a neural network model and train
# define base model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(13, input_dim=72, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [46]:
# define the model
def larger_model():
	# create model
	model = Sequential()
	model.add(Dense(13, input_dim=72, kernel_initializer='normal', activation='relu'))
	model.add(Dense(6, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [48]:
# define wider model
def wider_model():
	# create model
	model = Sequential()
	model.add(Dense(20, input_dim=72, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [36]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [41]:
# evaluate model
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X_train, y_train, cv=kfold)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

C:\Users\Revi\AppData\Local\Temp/ipykernel_17384/3031099890.py:2: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)


Baseline: -129138374515702989062144.00 (104434242413778824069120.00) MSE


In [47]:
#Boosting the performance by standardizing the inputs
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X_train, y_train, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

C:\Users\Revi\AppData\Local\Temp/ipykernel_17384/252686729.py:5: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))


Larger: -129139144406059284692992.00 (104434460836231588610048.00) MSE


In [49]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X_train, y_train, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

C:\Users\Revi\AppData\Local\Temp/ipykernel_17384/2925588626.py:3: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=100, batch_size=5, verbose=0)))


Wider: -129139150936278741024768.00 (104434457654033125474304.00) MSE


In [61]:
def model_eval(obs,pred):

    import numpy as np
    from sklearn import linear_model
    
    if obs.shape != pred.shape:
        print('flattening arrays')
        obs = obs.flatten()
        pred = pred.flatten()
        
    # Handle NaNs and Infs - ADDED RMM 8.3.2020 to handle NaNs and Infs
    nan_pred_mask = np.isnan(pred) | np.isinf(pred)
    if np.any(nan_pred_mask):
        not_nan_pred_mask = ~nan_pred_mask
        pred = pred[not_nan_pred_mask]
        obs = obs[not_nan_pred_mask]
        
#     from sklearn import metrics
    MAE = 1./len(obs) * (np.sum( np.abs(obs - pred) ))
#     MAE = metrics.mean_absolute_error(obs,pred)
    MSE = 1./len(obs) * (np.sum( (obs - pred)**2 ))
#     MSE = metrics.mean_squared_error(obs,pred)
    RMSE = np.sqrt(MSE)
    RAE = (np.sum( np.abs(obs - pred) )) / (np.sum( np.abs(obs - np.mean(obs)) ))
    RSE = (np.sum( (obs - pred)**2 )) / (np.sum( (obs - np.mean(obs))**2 ))
    RSQ = 1-RSE
#     RSQ = metrics.r2_score(obs,pred)
#     NOTE: Prediction efficiency and R-squared are the same thing
#     PE = 1 - ( (np.sum(pred - obs))**2 ) / ( (np.sum(obs - np.nanmean(obs)))**2 )
    #regr = linear_model.LinearRegression()
    #regr.fit(obs.reshape(-1, 1),pred)
    #lin_slope = regr.coef_[0]
   # lin_intercept = regr.intercept_
    

    print('Evaluation: \n')
    print('   MAE  = {}'.format(MAE))
    print('   MSE  = {}'.format(MSE))
    print('   RMSE = {}'.format(RMSE))
    print('   RAE  = {}'.format(RAE))
    print('   RSE  = {}'.format(RSE))
    print('   RSQ  = {}'.format(RSQ))
    return MAE, MSE, RMSE, RAE, RSE, RSQ
#     print('   PE  = {}'.format(PE))
   # print('   linear slope  = {}'.format(lin_slope))
   # print('   linear intercept  = {}'.format(lin_intercept))
    
   # return MAE, MSE, RMSE, RAE, RSE, RSQ  #, lin_slope, lin_intercept
   

In [54]:
y_preds.shape

(26,)

In [55]:
y_val.shape

(26,)

In [62]:
# Apply model to predict
pipeline.fit(X_train, y_train)
y_preds = pipeline.predict(X_val)
model_eval(y_val,y_preds)

Evaluation: 

   MAE  = 79790651345.9144
   MSE  = 4.043879503910716e+22
   RMSE = 201093995532.2067
   RAE  = 0.7242210009086224
   RSE  = 1.1868543787612056
   RSQ  = -0.1868543787612056


(79790651345.9144,
 4.043879503910716e+22,
 201093995532.2067,
 0.7242210009086224,
 1.1868543787612056,
 -0.1868543787612056)

In [ ]:
X_train_scaled

In [63]:
# Executing the model on scaLed model
pipeline.fit(X_train_scaled, y_train_log)
y_preds = pipeline.predict(X_val_scaled)
model_eval(y_val_log,y_preds)

Evaluation: 

   MAE  = 1.4538664844072735
   MSE  = 2.7608528724484946
   RMSE = 1.661581437200264
   RAE  = 0.9955320085930222
   RSE  = 1.145336666099887
   RSQ  = -0.14533666609988694


(1.4538664844072735,
 2.7608528724484946,
 1.661581437200264,
 0.9955320085930222,
 1.145336666099887,
 -0.14533666609988694)

In [81]:
def PreProcessData(df):
    
    # Remove outliers
    threshold_drop_val = np.percentile(df['ELE_TOTAL_ENERGY_FLUX'],eflux_threshold_quantile)
    print('... dropping rows where total electron energy flux is greater than {:.4E} [eV/cm2/s/ster]'.format(threshold_drop_val))
    print('--------> number dropped = {}'.format(len( np.argwhere(df['ELE_TOTAL_ENERGY_FLUX'].values > threshold_drop_val) )))
    df = df[ (df['ELE_TOTAL_ENERGY_FLUX'].values < threshold_drop_val) ]
    print(df.shape)


    # Separate training and testing data
    mask_val = [(df.index.year == 2010) & (df['ID_SC'].values==16)]
    df_val = df[mask_val[0]].copy(deep=True)
    df_train = df.copy(deep=True).drop( df.index[mask_val[0]])
    print('validation data shape = {}'.format(df_val.shape))
    print('train data shape = {}'.format(df_train.shape))
    print('NOTE: we have called the withheld data *validation* data here')


    # Scale and Transform inputs
    X_val = df_val[feature_cols].copy(deep=True)
    y_val = df_val['ELE_TOTAL_ENERGY_FLUX'].copy(deep=True)
    X_train = df_train[feature_cols].copy(deep=True)
    y_train = df_train['ELE_TOTAL_ENERGY_FLUX'].copy(deep=True)
    scaler_X = RobustScaler()
    scaler_X = scaler_X.fit(X_train.values)
    X_val_scaled = scaler_X.transform(X_val.values)
    X_train_scaled = scaler_X.transform(X_train.values)

    # Scale and Transform labels
    # The units of the energy flux are eV/cm2/s/ster
    #   --> To remove the 'steradian' multiply by PI (see: https://agupubs.onlinelibrary.wiley.com/doi/epdf/10.1002/2016JA023339)
    y_train = y_train * np.pi
    y_val = y_val * np.pi
    print('NOTE: removing the steradian from the DMSP data BEFORE training')
    # Create targets in erg/cm2/s units to explore target in different units
    y_train_erg = y_train.copy(deep=True) * (1.60218e-12)
    y_val_erg = y_val.copy(deep=True) * (1.60218e-12)
    # Create targets in log10(eV/cm2/s) units to explore target in different units
    y_train[y_train == 0] = 0.00001
    y_val[y_val == 0] = 0.00001
    y_train_log = np.log10(y_train.copy(deep=True))
    y_val_log = np.log10(y_val.copy(deep=True))
    
    return X_train_scaled, X_val_scaled,y_train_log, y_val_log


In [97]:
def CreateAndTrainModel():
    # Defining Checkpoints fr the model 
    checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
    file_name = os.path.join("./model/",checkpoint_name)
    checkpoint = ModelCheckpoint(file_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
    callback_ES = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    callback_TB = tf.keras.callbacks.TensorBoard(log_dir='./logs')
    callbacks_list = [checkpoint,callback_ES,callback_TB]

    # create model
    model = Sequential()
    model.add(Dense(20, input_dim=72, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='sgd')
    #estimators = []
    #estimators.append(('standardize', StandardScaler()))
    #estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=100, batch_size=5, verbose=0, callbacks=callbacks_list)))
    #pipeline = Pipeline(estimators)
    #return pipeline
    history = model.fit(X_train_scaled,y_train_log, epochs=100,validation_data=(X_val_scaled,y_val_log),callbacks=callbacks_list)
    return model


In [67]:
def TrainModel(pipeline,X_train_scaled,X_val_scaled,y_val_log):
    # Executing the model on scaLed model
    pipeline.fit(X_train_scaled, y_train_log)
    y_preds = pipeline.predict(X_val_scaled)
    return y_preds

In [68]:
def ValidateModel(y_val_log,y_preds):
    # Validate the model
    model_eval(y_val_log,y_preds)

In [98]:
#Training the model for entire dataset

df = df_tmp.copy()
X_train_scaled, X_val_scaled,y_train_log, y_val_log = PreProcessData(df)

... dropping rows where total electron energy flux is greater than 1.2880E+13 [eV/cm2/s/ster]
--------> number dropped = 98
(1945691, 73)
validation data shape = (55208, 73)
train data shape = (1836963, 73)
NOTE: we have called the withheld data *validation* data here
NOTE: removing the steradian from the DMSP data BEFORE training


In [107]:
y_val_log.dtype
actual = np.array(y_val_log)

In [116]:
actual.shape

(55208,)

In [117]:
predicted = y_preds.reshape(55208,)

In [118]:
predicted

array([1.        , 1.        , 1.        , ..., 0.99999976, 0.99999976,
       0.99999976], dtype=float32)

In [99]:
# Create a model
model = CreateAndTrainModel()


# Train the model
y_preds = model.predict(X_val_scaled)

# Validate the model
#ValidateModel(y_val_log,y_preds)

mse_test = model.evaluate(X_val_scaled,y_val_log)
print(mse_test)


Epoch 1/100
57400/57406 [============================>.] - ETA: 0s - loss: 75.7840
Epoch 1: val_loss improved from inf to 71.51799, saving model to ./model\Weights-001--71.51799.hdf5
57406/57406 [==============================] - 155s 3ms/step - loss: 75.7842 - val_loss: 71.5180
Epoch 2/100
57405/57406 [============================>.] - ETA: 0s - loss: 75.7780
Epoch 2: val_loss improved from 71.51799 to 71.51788, saving model to ./model\Weights-002--71.51788.hdf5
57406/57406 [==============================] - 157s 3ms/step - loss: 75.7780 - val_loss: 71.5179
Epoch 3/100
57395/57406 [============================>.] - ETA: 0s - loss: 75.7782
Epoch 3: val_loss improved from 71.51788 to 71.51785, saving model to ./model\Weights-003--71.51785.hdf5
57406/57406 [==============================] - 152s 3ms/step - loss: 75.7780 - val_loss: 71.5178
Epoch 4/100
57390/57406 [============================>.] - ETA: 0s - loss: 75.7776
Epoch 4: val_loss improved from 71.51785 to 71.51783, saving model 

In [120]:
y_val_log.shape

(55208,)

In [121]:
y_preds.shape


(55208, 1)

In [122]:
ValidateModel(actual,predicted)

Evaluation: 

   MAE  = 8.317060460472954
   MSE  = 71.51790967915615
   RMSE = 8.456826217864249
   RAE  = 6.009720632038825
   RSE  = 30.505655717542275
   RSQ  = -29.505655717542275


In [134]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

In [132]:
X_val_scaled.dtype

dtype('float64')

In [144]:
from sklearn.model_selection import train_test_split
df = df_tmp.sample(1000)
X_train_scaled, X_val_scaled,y_train_log, y_val_log = PreProcessData(df)

#convert y values to categorical values
lab = preprocessing.LabelEncoder()
y_transformed = lab.fit_transform(y_val_log)

model1 = LogisticRegression()
rfe = RFE(model1,n_features_to_select=10)
fit = rfe.fit(X_val_scaled, y_transformed)
#print("Num Features: %d") % fit.n_features_
#print("Selected Features: %s") % fit.support_
#print("Feature Ranking: %s") % fit.ranking_
print(fit.n_features_)
print(fit.support_)
print(fit.ranking_)

... dropping rows where total electron energy flux is greater than 4.2938E+12 [eV/cm2/s/ster]
--------> number dropped = 1
(999, 73)
validation data shape = (27, 73)
train data shape = (972, 73)
NOTE: we have called the withheld data *validation* data here
NOTE: removing the steradian from the DMSP data BEFORE training
10
[False False False False False False False False False False False False
 False False False False False False False False  True  True False False
 False  True  True  True  True False False False False False False False
 False False False False False False False False  True False  True False
  True False False False False False False False False False False False
 False False False  True False False False False False False False False]
[17 63 10 26  6 19 21 15 62 11 20 33 35  2 34 45 18 13 28 36  1  1  9  7
  3  1  1  1  1 27  8 23 52 38 37 48 16 12 42 44 56 39 61 30  1 40  1 55
  1 60 49 59 43 25 24 31 51 22 53 57 58 50 46  1  5 47 14 41 54 29  4 32]


In [145]:
df.columns

Index(['SC_AACGM_LAT', 'ELE_TOTAL_ENERGY_FLUX', 'ID_SC', 'sin_ut', 'cos_ut',
       'sin_doy', 'cos_doy', 'sin_SC_AACGM_LTIME', 'cos_SC_AACGM_LTIME',
       'F107', 'Bz', 'By', 'Bx', 'AL', 'AU', 'SymH', 'PC', 'vsw', 'psw',
       'Bz_6hr', 'By_6hr', 'Bx_6hr', 'AL_6hr', 'AU_6hr', 'SymH_6hr', 'PC_6hr',
       'vsw_6hr', 'psw_6hr', 'Bz_3hr', 'By_3hr', 'Bx_3hr', 'AL_3hr', 'AU_3hr',
       'SymH_3hr', 'PC_3hr', 'vsw_3hr', 'psw_3hr', 'Bz_1hr', 'By_1hr',
       'Bx_1hr', 'AL_1hr', 'AU_1hr', 'SymH_1hr', 'PC_1hr', 'vsw_1hr',
       'psw_1hr', 'Bz_45min', 'By_45min', 'Bx_45min', 'AL_45min', 'AU_45min',
       'SymH_45min', 'PC_45min', 'vsw_45min', 'psw_45min', 'Bz_30min',
       'By_30min', 'Bx_30min', 'AL_30min', 'AU_30min', 'SymH_30min',
       'PC_30min', 'vsw_30min', 'psw_30min', 'Bz_10min', 'By_10min',
       'Bx_10min', 'AL_10min', 'AU_10min', 'SymH_10min', 'PC_10min',
       'vsw_10min', 'psw_10min'],
      dtype='object')

In [147]:
fit.support_.size

72

In [149]:
df.columns.drop('ELE_TOTAL_ENERGY_FLUX')

Index(['SC_AACGM_LAT', 'ID_SC', 'sin_ut', 'cos_ut', 'sin_doy', 'cos_doy',
       'sin_SC_AACGM_LTIME', 'cos_SC_AACGM_LTIME', 'F107', 'Bz', 'By', 'Bx',
       'AL', 'AU', 'SymH', 'PC', 'vsw', 'psw', 'Bz_6hr', 'By_6hr', 'Bx_6hr',
       'AL_6hr', 'AU_6hr', 'SymH_6hr', 'PC_6hr', 'vsw_6hr', 'psw_6hr',
       'Bz_3hr', 'By_3hr', 'Bx_3hr', 'AL_3hr', 'AU_3hr', 'SymH_3hr', 'PC_3hr',
       'vsw_3hr', 'psw_3hr', 'Bz_1hr', 'By_1hr', 'Bx_1hr', 'AL_1hr', 'AU_1hr',
       'SymH_1hr', 'PC_1hr', 'vsw_1hr', 'psw_1hr', 'Bz_45min', 'By_45min',
       'Bx_45min', 'AL_45min', 'AU_45min', 'SymH_45min', 'PC_45min',
       'vsw_45min', 'psw_45min', 'Bz_30min', 'By_30min', 'Bx_30min',
       'AL_30min', 'AU_30min', 'SymH_30min', 'PC_30min', 'vsw_30min',
       'psw_30min', 'Bz_10min', 'By_10min', 'Bx_10min', 'AL_10min', 'AU_10min',
       'SymH_10min', 'PC_10min', 'vsw_10min', 'psw_10min'],
      dtype='object')

In [154]:
column1 = pd.Series(df.columns.drop('ELE_TOTAL_ENERGY_FLUX'))
column2 = pd.Series(fit.support_)
column3 = pd.Series(fit.ranking_)

In [155]:
feature_df = pd.DataFrame ({ 'Feature': column1,  'Selected': column2, 'Rank': column3})

In [156]:
feature_df

,Feature,Selected,Rank
0,SC_AACGM_LAT,False,17
1,ID_SC,False,63
2,sin_ut,False,10
3,cos_ut,False,26
4,sin_doy,False,6
...,...,...,...
67,AU_10min,False,41
68,SymH_10min,False,54
69,PC_10min,False,29
70,vsw_10min,False,4


In [157]:
rslt_df = feature_df.loc[feature_df['Selected'] == True] 
rslt_df

,Feature,Selected,Rank
20,Bx_6hr,True,1
21,AL_6hr,True,1
25,vsw_6hr,True,1
26,psw_6hr,True,1
27,Bz_3hr,True,1
28,By_3hr,True,1
44,psw_1hr,True,1
46,By_45min,True,1
48,AL_45min,True,1
63,Bz_10min,True,1


In [160]:
selected_features = rslt_df['Feature']
selected_features = pd.Series(selected_features)

In [173]:
selected_features
feature_cols = list(selected_features)
feature_cols

['Bx_6hr',
 'AL_6hr',
 'vsw_6hr',
 'psw_6hr',
 'Bz_3hr',
 'By_3hr',
 'psw_1hr',
 'By_45min',
 'AL_45min',
 'Bz_10min']

In [171]:
key_features=['Bx_6hr','AL_6hr','vsw_6hr','psw_6hr','Bz_3hr','By_3hr','psw_1hr','By_45min','AL_45min','Bz_10min','ELE_TOTAL_ENERGY_FLUX','ID_SC']


df_new = df_tmp[key_features]


In [172]:
df_new.head(10)

,Bx_6hr,AL_6hr,vsw_6hr,psw_6hr,Bz_3hr,By_3hr,psw_1hr,By_45min,AL_45min,Bz_10min,ELE_TOTAL_ENERGY_FLUX,ID_SC
Datetimes,,,,,,,,,,,,
1987-01-12 12:57:00,-2.877500,-18.750000,337.700000,3.977500,3.435000,9.126667,4.765000,6.06,-107.0,1.25,2.057827e+08,6.0
1987-01-12 12:58:00,-2.877500,-18.750000,337.700000,3.977500,3.435000,9.126667,4.765000,6.06,-107.0,1.25,2.509837e+08,6.0
1987-01-12 13:01:00,-2.364167,-14.500000,338.100000,4.078333,2.738333,9.205000,4.770000,5.58,-92.0,1.51,4.469146e+08,6.0
1987-01-12 13:02:00,-2.364167,-14.500000,338.100000,4.078333,2.738333,9.205000,4.770000,5.58,-92.0,1.51,1.934849e+10,6.0
1987-01-12 13:03:00,-2.364167,-14.500000,338.100000,4.078333,2.738333,9.205000,4.770000,5.58,-92.0,1.51,3.913337e+11,6.0
1987-01-12 13:04:00,-2.364167,-14.500000,338.100000,4.078333,2.738333,9.205000,4.770000,5.58,-92.0,1.51,8.838582e+10,6.0
1987-01-28 19:08:00,1.524167,-120.333333,438.308333,3.158333,-2.400000,-3.436667,3.278333,3.10,-455.0,6.67,6.824302e+07,6.0
1987-01-28 19:09:00,1.524167,-120.333333,438.308333,3.158333,-2.400000,-3.436667,3.278333,3.10,-455.0,6.67,1.114998e+08,6.0
1987-01-28 19:10:00,1.181667,-118.916667,435.833333,3.231667,-1.720000,-0.915000,3.153333,3.07,-455.0,6.44,1.397239e+08,6.0


In [179]:
def CreateAndTrainOptModel():
    # Defining Checkpoints fr the model 
    checkpoint_name = 'Optimal-Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
    file_name = os.path.join("./model/",checkpoint_name)
    checkpoint = ModelCheckpoint(file_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
    callback_ES = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    callback_TB = tf.keras.callbacks.TensorBoard(log_dir='./logs')
    callbacks_list = [checkpoint,callback_ES,callback_TB]

    # create model
    model = Sequential()
    model.add(Dense(20, input_dim=10, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='sgd')
    #estimators = []
    #estimators.append(('standardize', StandardScaler()))
    #estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=100, batch_size=5, verbose=0, callbacks=callbacks_list)))
    #pipeline = Pipeline(estimators)
    #return pipeline
    history = model.fit(X_train_scaled,y_train_log, epochs=100,validation_data=(X_val_scaled,y_val_log),callbacks=callbacks_list)
    return model


In [178]:
df_new.columns

Index(['Bx_6hr', 'AL_6hr', 'vsw_6hr', 'psw_6hr', 'Bz_3hr', 'By_3hr', 'psw_1hr',
       'By_45min', 'AL_45min', 'Bz_10min', 'ELE_TOTAL_ENERGY_FLUX', 'ID_SC'],
      dtype='object')

In [181]:
# Creating a simple model with 10 features

X_train_scaled, X_val_scaled,y_train_log, y_val_log = PreProcessData(df_new)
# Create a model
model1 = CreateAndTrainOptModel()

# Train the model
y_preds = model1.predict(X_val_scaled)

mse_test = model1.evaluate(X_val_scaled,y_val_log)
print(mse_test)

actual = np.array(y_val_log)
predicted = y_preds.reshape(55208,)

ValidateModel(actual,predicted)

... dropping rows where total electron energy flux is greater than 1.2880E+13 [eV/cm2/s/ster]
--------> number dropped = 98
(1945691, 12)
validation data shape = (55208, 12)
train data shape = (1836963, 12)
NOTE: we have called the withheld data *validation* data here
NOTE: removing the steradian from the DMSP data BEFORE training
Epoch 1/100
57389/57406 [============================>.] - ETA: 0s - loss: 75.7856
Epoch 1: val_loss improved from inf to 71.51794, saving model to ./model\Optimal-Weights-001--71.51794.hdf5
57406/57406 [==============================] - 125s 2ms/step - loss: 75.7853 - val_loss: 71.5179
Epoch 2/100
57397/57406 [============================>.] - ETA: 0s - loss: 75.7785
Epoch 2: val_loss improved from 71.51794 to 71.51785, saving model to ./model\Optimal-Weights-002--71.51785.hdf5
57406/57406 [==============================] - 126s 2ms/step - loss: 75.7784 - val_loss: 71.5179
Epoch 3/100
57393/57406 [============================>.] - ETA: 0s - loss: 75.7778
Epo

In [ ]:
# Helper function for plotting feature importance
def plot_features(columns, importances, n=20):
    df = (pd.DataFrame({"features": columns,
                        "feature_importance": importances})
          .sort_values("feature_importance", ascending=False)
          .reset_index(drop=True))
    
    sns.barplot(x="feature_importance",
                y="features",
                data=df[:n],
                orient="h")

In [ ]:
plot_features(X.columns, fs.scores_)